In [3]:
import shutil
import os
from spreco.common import utils,ops 
import numpy as np
from numpy.fft import fftshift, ifftshift, ifft2,fft2

import h5py
from tqdm import tqdm
import glob

def sos(x, axis=- 1):
    '''
    root mean sum of squares
    '''
    res = np.sqrt(np.sum(np.abs(x)**2, axis=axis))
    return res
def crop(x, wout, hout):
    w, h = x.shape[2:]
    x1 = int(np.ceil((w - wout) / 2.))
    y1 = int(np.ceil((h - hout) / 2.))
    return x[:,:,x1:x1+wout, y1:y1+hout]

def zpad(array_in, outshape):
    import math
    #out = np.zeros(outshape, dtype=array_in.dtype)
    oldshape = array_in.shape
    assert len(oldshape)==len(outshape)
    #kspdata = np.array(kspdata)
    pad_list=[]
    for iold, iout in zip(oldshape, outshape):
        left = math.floor((iout-iold)/2)
        right = math.ceil((iout-iold)/2)
        pad_list.append((left, right))

    zfill = np.pad(array_in, pad_list, 'constant')                     # fill blade into square with 0
    return zfill


def kspace_correct(matrix):
    # 找到矩阵中的最大值的索引
    max_index = np.unravel_index(np.argmax(matrix), matrix.shape)
    # 计算中心位置的偏移
    center_offset = np.array(matrix.shape[1:]) // 2 - np.array(max_index[1:])

    print(center_offset)
    shifted_matrix = np.roll(matrix, center_offset, axis=(1,2))
    # 输出结果
    return shifted_matrix

In [4]:
paths = sorted(glob.glob(f'ori_h5/*T2*.h5'))

for path in paths:

    name = path.split('/')[-1].replace('h5','npy')

    kspace = np.array(h5py.File(path, "r")['kspace'])
    
    for i in range(kspace.shape[0]):
        kspace[i] = kspace_correct(kspace[i])

    nz, c, nx, ny = kspace.shape
    print(kspace.shape)
    img_shape = [nx, ny]
    img = ops.mifft2(kspace, dims=img_shape,axis=(2,3))

    nz, c, nx, ny = kspace.shape
    
    
    if ny == 396: ## fastMRI T2有396的sacle，直接crop到384。方便UNet网络
        ny = 384
    
    img = crop(img,ny,ny)
    kspace = ops.mfft2(img, dims=[ny,ny],axis=(2,3))

    kspace = kspace.transpose(0,2,3,1)
    
    print(kspace.shape)
    if ny < 384:
        kspace = zpad(kspace,(nz,384,384,c))
    print(kspace.shape)

    coilsens = np.zeros_like(kspace, dtype='complex64')
    for i in range(kspace.shape[0]):
        coilsens[i] = utils.bart(1, 'ecalib -m1 -r20 -c0.001', kspace[i][None])[0]
        
    masks = []
    coilsens_985 = np.zeros_like(kspace, dtype='complex64')
    for i in range(kspace.shape[0]):
        tmp_coilsen = np.squeeze(utils.bart(1, 'ecalib -m1 -r20 -c0.985', kspace[i][None]))
        coilsens_985[i] = tmp_coilsen
        masks.append(np.abs(tmp_coilsen[...,0]) > 0)
    
    rss = []
    for slice in range(kspace.shape[0]):    
        tmp    = np.sum(np.conj(coilsens[slice]) * ops.mifft2(kspace[slice], dims=(ny,ny),axis=(0,1)),-1) 
        rss.append(tmp)
    rss = np.abs(np.array(rss)).astype(np.float32)

    sample = {
        'kspace': np.array(kspace,dtype=np.complex64),
        'csm_001': np.array(coilsens,dtype=np.complex64),
        'csm_985': np.array(coilsens_985,dtype=np.complex64),
        'mask': np.array(masks,dtype=np.uint8),
        'rss': rss,
    }

    np.save(f'raw/{name}',sample)

[ 0 -1]
[ 0 -1]
[ 0 -1]
[ 0 -1]
[ 0 -1]
[ 1 -1]
[ 1 -1]
[ 1 -1]
[ 1 -1]
[ 1 -1]
[ 0 -1]
[ 0 -1]
[ 0 -1]
[ 0 -1]
[0 0]
[-1 -1]
(16, 16, 768, 396)
(16, 384, 384, 16)
(16, 384, 384, 16)
bart ecalib -m1 -r20 -c0.001 /tmp/tmprizotd6tin0 /tmp/tmprizotd6tout0
Done.
bart ecalib -m1 -r20 -c0.001 /tmp/tmph5q1305cin0 /tmp/tmph5q1305cout0
Make sureyou install bart properly


Exception: Command exited with an error.